<a href="https://colab.research.google.com/github/seunghyeon2776/98/blob/master/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dataset

In [ ]:
import os
import glob
import numpy as np
import cv2
from sklearn.utils import shuffle

def load_train(train_path, image_size, classes):
    images = []
    labels = []
    ids = []
    cls = []

    print('Reading training images')
    for fld in classes:
        index = classes.index(fld)
        print('Loading {} files (Index: {})'.format(fld, index))
        path = os.path.join(train_path, fld, '*g')
        files = glob.glob(path)
        for fl in files:
            image = cv2.imread(fl)
            image = cv2.resize(image, (image_size, image_size), cv2.INTER_LINEAR)
            images.append(image)
            label = np.zeros(len(classes))
            label[index] = 1.0
            labels.append(label)
            flbase = os.path.basename(fl)
            ids.append(flbase)
            cls.append(fld)
    images = np.array(images)
    labels = np.array(labels)
    ids = np.array(ids)
    cls = np.array(cls)
  

    return images, labels, ids, cls


def load_test(test_path, image_size):
    path = os.path.join(test_path, '*g')
    files = sorted(glob.glob(path))

    X_test = []
    X_test_id = []
    print("Reading test images")
    for fl in files:
        
        img = cv2.imread(fl)
        img = cv2.resize(img, (image_size, image_size), cv2.INTER_LINEAR)
        X_test.append(img)
        X_test_id.append(flbase)
        flbase = os.path.basename(fl)

    X_test = np.array(X_test, dtype=np.uint8)
    X_test = X_test.astype('float32')
    X_test = X_test / 255
    X_test_id = np.array(X_test_id)

    return X_test, X_test_id


class DataSet(object):

    def __init__(self, images, labels, ids, cls):
        """Construct a DataSet. one_hot arg is used only if fake_data is true."""

        self._num_examples = images.shape[0]

        # Convert shape from [num examples, rows, columns, depth]
        # to [num examples, rows*columns] (assuming depth == 1)
        # Convert from [0, 255] -> [0.0, 1.0].

        images = images.astype(np.float32)
        images = np.multiply(images, 1.0 / 255.0)

        self._images = images
        self._labels = labels
        self._ids = ids
        self._cls = cls
        self._epochs_completed = 0
        self._index_in_epoch = 0

    @property
    def images(self):
        return self._images

    @property
    def labels(self):
        return self._labels

    @property
    def ids(self):
        return self._ids

    @property
    def cls(self):
        return self._cls

    @property
    def num_examples(self):
        return self._num_examples

    @property
    def epochs_completed(self):
        return self._epochs_completed

    def next_batch(self, batch_size):
        """Return the next `batch_size` examples from this data set."""
        start = self._index_in_epoch
        self._index_in_epoch += batch_size

        if self._index_in_epoch > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1

            # # Shuffle the data (maybe)
            # perm = np.arange(self._num_examples)
            # np.random.shuffle(perm)
            # self._images = self._images[perm]
            # self._labels = self._labels[perm]
            # Start next epoch

            start = 0
            self._index_in_epoch = batch_size
            assert batch_size <= self._num_examples
        end = self._index_in_epoch

        return self._images[start:end], self._labels[start:end], self._ids[start:end], self._cls[start:end]


def read_train_sets(train_path, image_size, classes, validation_size=0):
    class DataSets(object):
        pass

    data_sets = DataSets()

    images, labels, ids, cls = load_train(train_path, image_size, classes)
    images, labels, ids, cls = shuffle(images, labels, ids, cls)  # shuffle the data

    if isinstance(validation_size, float):
        validation_size = int(validation_size * images.shape[0])

    train_images = images
    train_labels = labels
    train_ids = ids
    train_cls = cls

    data_sets.train = DataSet(train_images, train_labels, train_ids, train_cls)

    return data_sets

In [ ]:
def load_validation(validation_path, image_size, classes):
    images2 = []
    labels2 = []
    ids2 = []
    cls2 = []

    print('Reading validation images')
    for fld in classes:
        index = classes.index(fld)
        print('Loading {} files (Index: {})'.format(fld, index))
        path = os.path.join(validation_path, fld, '*g')
        files = glob.glob(path)
        for fl in files:
            image = cv2.imread(fl)
            image = cv2.resize(image, (image_size, image_size), cv2.INTER_LINEAR)
            images2.append(image)
            label = np.zeros(len(classes))
            label[index] = 1.0
            labels2.append(label)
            flbase = os.path.basename(fl)
            ids2.append(flbase)
            cls2.append(fld)
    images2 = np.array(images2)
    labels2 = np.array(labels2)
    ids2 = np.array(ids2)
    cls2 = np.array(cls2)
  

    return images2, labels2, ids2, cls2


def read_validation_sets(validation_path, image_size, classes, validation_size=0):
    class DataSets(object):
        pass

    data_sets = DataSets()

    images2, labels2, ids2, cls2 = load_validation(validation_path, image_size, classes)
    images2, labels2, ids2, cls2 = shuffle(images2, labels2, ids2, cls2)  # shuffle the data

    if isinstance(validation_size, float):
        validation_size = int(validation_size * images2.shape[0])

    validation_images = images2
    validation_labels = labels2
    validation_ids = ids2
    validation_cls = cls2

    data_sets.valid = DataSet(validation_images, validation_labels, validation_ids, validation_cls)

    return data_sets

def load_test(test_path, image_size, classes):
    images3 = []
    labels3 = []
    ids3 = []
    cls3 = []

    print('Reading validation images')
    for fld in classes:
        index = classes.index(fld)
        print('Loading {} files (Index: {})'.format(fld, index))
        path = os.path.join(test_path, fld, '*g')
        files = glob.glob(path)
        for fl in files:
            image = cv2.imread(fl)
            image = cv2.resize(image, (image_size, image_size), cv2.INTER_LINEAR)
            images3.append(image)
            label = np.zeros(len(classes))
            label[index] = 1.0
            labels3.append(label)
            flbase = os.path.basename(fl)
            ids3.append(flbase)
            cls3.append(fld)
    images3 = np.array(images3)
    labels3 = np.array(labels3)
    ids3 = np.array(ids3)
    cls3 = np.array(cls3)
  

    return images3, labels3, ids3, cls3


def read_test_sets(test_path, image_size, classes, validation_size=0):
    class DataSets(object):
        pass

    data_sets = DataSets()

    images3, labels3, ids3, cls3 = load_test(test_path, image_size, classes)
    images3, labels3, ids3, cls3 = shuffle(images3, labels3, ids3, cls3)  # shuffle the data

    if isinstance(validation_size, float):
        validation_size = int(validation_size * images3.shape[0])

    test_images = images3
    test_labels = labels3
    test_ids = ids3
    test_cls = cls3

    data_sets.test = DataSet(test_images, test_labels, test_ids, test_cls)

    return data_sets

In [ ]:
import time
import math
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
import cv2
import dataset
import os
import keras

from sklearn.metrics import confusion_matrix
from datetime import timedelta
import seaborn as sn

%matplotlib inline

tf.disable_v2_behavior()

filter_size0 = 3
num_filters0 = 3

filter_size1 = 3
num_filters1 = 64

filter_size2 = 3
num_filters2 = 128

filter_size3 = 3
num_filters3 = 256

filter_size4 = 3
num_filters4 = 512

filter_size5 = 3
num_filters5 = 512

filter_size6 = 7
num_filters6 = 512

filter_size7 = 3
num_filters7 = 1024

filter_size8 = 3
num_filters8 = 2048

# Fully-connected layer.
fc_size = 4096
fc_size2 = 1000    
fc_size3 = 64       

# Number of color channels for the images: 1 channel for gray-scale.
num_channels = 3

# image dimensions (only squares for now)
img_size = 150

# Size of image when flattened to a single dimension
img_size_flat = img_size * img_size * num_channels

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# class info
classes = ['1.Cancer', '2.Precancer', '3.Inflammatory', '4.Normal']
num_classes = len(classes)

# batch size
batch_size = 32

# validation split
validation_size = 0

# how long to wait after validation loss stops improving before terminating training
early_stopping = None

In [ ]:
train_path = '/content/drive/Shareddrives/CTRC-OralDetect-Project/NewDataGenerate-HM/train'
validation_path = '/content/drive/Shareddrives/CTRC-OralDetect-Project/NewDataGenerate-HM/validation'
test_path = '/content/drive/Shareddrives/CTRC-OralDetect-Project/NewDataGenerate-HM/test'
checkpoint_dir = '/content/drive/MyDrive/model2/'

data = read_train_sets(train_path, img_size, classes, validation_size=validation_size)
data2 = read_validation_sets(validation_path, img_size, classes, validation_size=validation_size)
data3 = read_test_sets(test_path, img_size, classes, validation_size=validation_size)


print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Validation-set:\t{}".format(len(data2.valid.labels)))
print("- Test-set:\t\t{}".format(len(data3.test.labels)))

In [ ]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

def new_conv_layer_2048(input,              
                   num_input_channels, 
                   filter_size,       
                   num_filters,       
                   use_pooling=False): 

    shape = [filter_size, filter_size, num_input_channels, num_filters]

    weights = new_weights(shape=shape)

    biases = new_biases(length=num_filters)

    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    layer += biases

    if use_pooling:

        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    layer = tf.nn.relu(layer)

    return layer, weights



def new_conv_layer_2048_2(input,              
                   num_input_channels, 
                   filter_size,       
                   num_filters,       
                   use_pooling=True): 

    shape = [filter_size, filter_size, num_input_channels, num_filters]

    weights = new_weights(shape=shape)

    biases = new_biases(length=num_filters)

    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    layer += biases

    if use_pooling:

        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    layer = tf.nn.relu(layer)

    return layer, weights



def new_conv_layer_1024(input,              
                   num_input_channels, 
                   filter_size,       
                   num_filters,       
                   use_pooling=False): 

    shape = [filter_size, filter_size, num_input_channels, num_filters]

    weights = new_weights(shape=shape)

    biases = new_biases(length=num_filters)

    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    layer += biases  

    if use_pooling:

        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    layer = tf.nn.relu(layer)

    return layer, weights



def new_conv_layer_512(input,              
                   num_input_channels, 
                   filter_size,       
                   num_filters,       
                   use_pooling=False): 

    shape = [filter_size, filter_size, num_input_channels, num_filters]

    weights = new_weights(shape=shape)

    biases = new_biases(length=num_filters)

    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    layer += biases  

    if use_pooling:

        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    layer = tf.nn.relu(layer)

    return layer, weights



def new_conv_layer_256(input,              
                   num_input_channels, 
                   filter_size,       
                   num_filters,       
                   use_pooling=False): 

    shape = [filter_size, filter_size, num_input_channels, num_filters]

    weights = new_weights(shape=shape)

    biases = new_biases(length=num_filters)

    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    layer += biases  

    if use_pooling:

        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    layer = tf.nn.relu(layer)

    return layer, weights



def new_conv_layer_128(input,              
                   num_input_channels, 
                   filter_size,       
                   num_filters,       
                   use_pooling=False): 

    shape = [filter_size, filter_size, num_input_channels, num_filters]

    weights = new_weights(shape=shape)

    biases = new_biases(length=num_filters)

    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    layer += biases  

    if use_pooling:

        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    layer = tf.nn.relu(layer)

    return layer, weights




def flatten_layer(layer):

    layer_shape = layer.get_shape()

    num_features = layer_shape[1:8].num_elements()
    
    layer_flat = tf.reshape(layer, [-1, num_features])

    return layer_flat, num_features





def new_fc_layer(input,        
                 num_inputs,    
                 num_outputs,  
                 use_relu=True):

    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    layer = tf.matmul(input, weights) + biases
    
    if use_relu:
        layer = tf.nn.relu(layer)
                
    return layer


def new_fc_layer2(input,        
                 num_inputs,    
                 num_outputs,  
                 use_relu=True):

    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    layer = tf.matmul(input, weights) + biases
    
    if use_relu:
        layer = tf.nn.relu(layer)
        layer = tf.nn.dropout(layer, keep_prob=keep_prob)
                
    return layer

x = tf.compat.v1.placeholder(tf.float32, shape=[None, img_size_flat], name='x')

x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])


y_true = tf.compat.v1.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

y_true_cls = tf.argmax(y_true, axis=1)

keep_prob = tf.compat.v1.placeholder(tf.float32)

In [ ]:
layer_conv1, weights_conv1 = \
    new_conv_layer_2048(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size8,
                   num_filters=num_filters8,
                   use_pooling=False)
     
layer_conv2, weights_conv2 = \
    new_conv_layer_2048_2(input=layer_conv1,
                   num_input_channels=num_filters8,
                   filter_size=filter_size8,
                   num_filters=num_filters8,
                   use_pooling=True)
    
layer_conv3, weights_conv3 = \
    new_conv_layer_1024(input=layer_conv2,
                   num_input_channels=num_filters8,
                   filter_size=filter_size7,
                   num_filters=num_filters7,
                   use_pooling=False)
    
layer_conv4, weights_conv4 = \
    new_conv_layer_1024(input=layer_conv3,
                   num_input_channels=num_filters7,
                   filter_size=filter_size7,
                   num_filters=num_filters7,
                   use_pooling=False)
    
layer_conv5, weights_conv5 = \
    new_conv_layer_1024(input=layer_conv4,
                   num_input_channels=num_filters7,
                   filter_size=filter_size7,
                   num_filters=num_filters7,
                   use_pooling=False)
    
layer_conv6, weights_conv6 = \
    new_conv_layer_1024(input=layer_conv5,
                   num_input_channels=num_filters7,
                   filter_size=filter_size7,
                   num_filters=num_filters7,
                   use_pooling=False)
    
layer_conv7, weights_conv7 = \
    new_conv_layer_1024(input=layer_conv6,
                   num_input_channels=num_filters7,
                   filter_size=filter_size7,
                   num_filters=num_filters7,
                   use_pooling=False)
    
layer_conv8, weights_conv8 = \
    new_conv_layer_1024(input=layer_conv7,
                   num_input_channels=num_filters7,
                   filter_size=filter_size7,
                   num_filters=num_filters7,
                   use_pooling=False)
    
layer_conv9, weights_conv9 = \
    new_conv_layer_1024(input=layer_conv8,
                   num_input_channels=num_filters7,
                   filter_size=filter_size7,
                   num_filters=num_filters7,
                   use_pooling=False)
    
layer_conv10, weights_conv10 = \
    new_conv_layer_512(input=layer_conv9,
                   num_input_channels=num_filters7,
                   filter_size=filter_size5,
                   num_filters=num_filters5,
                   use_pooling=False)
    
layer_conv11, weights_conv11 = \
    new_conv_layer_512(input=layer_conv10,
                   num_input_channels=num_filters5,
                   filter_size=filter_size5,
                   num_filters=num_filters5,
                   use_pooling=False)
    
layer_conv12, weights_conv12 = \
    new_conv_layer_512(input=layer_conv11,
                   num_input_channels=num_filters5,
                   filter_size=filter_size5,
                   num_filters=num_filters5,
                   use_pooling=False)
    
layer_conv13, weights_conv13 = \
    new_conv_layer_512(input=layer_conv12,
                   num_input_channels=num_filters5,
                   filter_size=filter_size5,
                   num_filters=num_filters5,
                   use_pooling=False)
    
layer_conv14, weights_conv14 = \
    new_conv_layer_512(input=layer_conv13,
                   num_input_channels=num_filters5,
                   filter_size=filter_size5,
                   num_filters=num_filters5,
                   use_pooling=False)
    
layer_conv15, weights_conv15 = \
    new_conv_layer_512(input=layer_conv14,
                   num_input_channels=num_filters5,
                   filter_size=filter_size5,
                   num_filters=num_filters5,
                   use_pooling=False)
    
layer_conv16, weights_conv16 = \
    new_conv_layer_512(input=layer_conv15,
                   num_input_channels=num_filters5,
                   filter_size=filter_size5,
                   num_filters=num_filters5,
                   use_pooling=False)
    
layer_conv17, weights_conv17 = \
    new_conv_layer_512(input=layer_conv16,
                   num_input_channels=num_filters5,
                   filter_size=filter_size5,
                   num_filters=num_filters5,
                   use_pooling=False)
    
layer_conv18, weights_conv18 = \
    new_conv_layer_512(input=layer_conv17,
                   num_input_channels=num_filters5,
                   filter_size=filter_size5,
                   num_filters=num_filters5,
                   use_pooling=False)
    
layer_conv19, weights_conv19 = \
    new_conv_layer_512(input=layer_conv18,
                   num_input_channels=num_filters5,
                   filter_size=filter_size5,
                   num_filters=num_filters5,
                   use_pooling=False)
    
layer_conv20, weights_conv20 = \
    new_conv_layer_512(input=layer_conv19,
                   num_input_channels=num_filters5,
                   filter_size=filter_size5,
                   num_filters=num_filters5,
                   use_pooling=False)
    
layer_conv21, weights_conv21 = \
    new_conv_layer_512(input=layer_conv20,
                   num_input_channels=num_filters5,
                   filter_size=filter_size5,
                   num_filters=num_filters5,
                   use_pooling=False)
    
layer_conv22, weights_conv22 = \
    new_conv_layer_256(input=layer_conv21,
                   num_input_channels=num_filters5,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv23, weights_conv23 = \
    new_conv_layer_256(input=layer_conv22,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv24, weights_conv24 = \
    new_conv_layer_256(input=layer_conv23,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv25, weights_conv25 = \
    new_conv_layer_256(input=layer_conv24,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv26, weights_conv26 = \
    new_conv_layer_256(input=layer_conv25,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv27, weights_conv27 = \
    new_conv_layer_256(input=layer_conv26,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv28, weights_conv28 = \
    new_conv_layer_256(input=layer_conv27,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv29, weights_conv29 = \
    new_conv_layer_256(input=layer_conv28,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv30, weights_conv30 = \
    new_conv_layer_256(input=layer_conv29,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv31, weights_conv31 = \
    new_conv_layer_256(input=layer_conv30,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv32, weights_conv32 = \
    new_conv_layer_256(input=layer_conv31,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv33, weights_conv33 = \
    new_conv_layer_256(input=layer_conv32,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv34, weights_conv34 = \
    new_conv_layer_256(input=layer_conv33,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv35, weights_conv35 = \
    new_conv_layer_256(input=layer_conv34,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv36, weights_conv36 = \
    new_conv_layer_256(input=layer_conv35,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv37, weights_conv37 = \
    new_conv_layer_256(input=layer_conv36,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv38, weights_conv38 = \
    new_conv_layer_256(input=layer_conv37,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv39, weights_conv39 = \
    new_conv_layer_256(input=layer_conv38,
                   num_input_channels=num_filters3,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
    
layer_conv40, weights_conv40 = \
    new_conv_layer_128(input=layer_conv39,
                   num_input_channels=num_filters3,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=False)
    
layer_conv41, weights_conv41 = \
    new_conv_layer_128(input=layer_conv40,
                   num_input_channels=num_filters2,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=False)
    
layer_conv42, weights_conv42 = \
    new_conv_layer_128(input=layer_conv41,
                   num_input_channels=num_filters2,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=False)
    
layer_conv43, weights_conv43 = \
    new_conv_layer_128(input=layer_conv42,
                   num_input_channels=num_filters2,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=False)
    
layer_conv44, weights_conv44 = \
    new_conv_layer_128(input=layer_conv43,
                   num_input_channels=num_filters2,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=False)
    
layer_conv45, weights_conv45 = \
    new_conv_layer_128(input=layer_conv44,
                   num_input_channels=num_filters2,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=False)
    
layer_conv46, weights_conv46 = \
    new_conv_layer_128(input=layer_conv45,
                   num_input_channels=num_filters2,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=False)
    
layer_flat, num_features = flatten_layer(layer_conv46)

layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=64,
                         use_relu=True)

layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=64,
                         num_outputs=num_channels,
                         use_relu=True)



y_pred = tf.nn.softmax(layer_fc2)

y_pred_cls = tf.argmax(y_pred, axis=1)

loss_func = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc2, labels=y_true)

cost = tf.reduce_mean(loss_func)

regularizer = tf.nn.l2_loss(weights_conv46)
beta = 0.001
cost = tf.reduce_mean(cost + beta*regularizer)

optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

correct_prediction = tf.equal(y_pred_cls, y_true_cls)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
session = tf.Session()

session.run(tf.global_variables_initializer())

train_batch_size = batch_size

def print_progress(epoch, feed_dict_train, val_loss):
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    msg = "Epoch {0} --- Training Accuracy: {1:>6.1%},  Validation Loss: {2:.3f}"
    print(msg.format(epoch + 1, acc, val_loss))

    if epoch <= 48:
      print(print_validation_accuracy(show_confusion_matrix=True))
      print(print_test_accuracy(show_confusion_matrix=True))
    else:
      print(print_validation_accuracy(show_confusion_matrix=True))
      print(print_test_accuracy(show_confusion_matrix=True))


total_iterations = 0

In [ ]:
def optimize(num_iterations):
    global total_iterations
    

    start_time = time.time()
    
    best_val_loss = float("inf")
    patience = 0

    for i in range(total_iterations,
                   total_iterations + num_iterations):

        

        x_batch, y_true_batch, _, cls_batch = data.train.next_batch(train_batch_size)
        x_valid_batch, y_valid_batch, _, valid_cls_batch = data2.valid.next_batch(train_batch_size)
        x_test_batch, y_test_batch, _, test_cls_batch = data3.test.next_batch(train_batch_size)

        x_batch = x_batch.reshape(train_batch_size, img_size_flat)
        x_valid_batch = x_valid_batch.reshape(train_batch_size, img_size_flat)
        x_test_batch = x_test_batch.reshape(train_batch_size, img_size_flat)

        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}
        
        feed_dict_validate = {x: x_valid_batch,
                              y_true: y_valid_batch}

        feed_dict_test = {x: x_test_batch,
                          y_true: y_test_batch}

        session.run(optimizer, feed_dict=feed_dict_train)
        
        if i % int(data.train.num_examples/batch_size) == 0: 
            val_loss = session.run(cost, feed_dict=feed_dict_validate)
            epoch = int(i / int(data.train.num_examples/batch_size))

            print_progress(epoch, feed_dict_train, val_loss)
            
            if early_stopping:    
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    patience = 0
                else:
                    patience += 1

                if patience == early_stopping:
                    break


    total_iterations += num_iterations

    end_time = time.time()

    time_dif = end_time - start_time

    print("Time elapsed: " + str(timedelta(seconds=int(round(time_dif)))))

In [ ]:
def print_validation_accuracy(show_example_errors=False, show_confusion_matrix=False):

    num_test = len(data2.valid.images)

    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    i = 0

    while i < num_test:
 
        j = min(i + batch_size, num_test)

        images = data2.valid.images[i:j, :].reshape(-1, img_size_flat)
        
        labels = data2.valid.labels[i:j, :]

        feed_dict = {x: images, y_true: labels}

        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        i = j


    cls_true = np.array(data2.valid.cls)
    cls_pred = np.array([classes[x] for x in cls_pred])

 

    correct = (cls_true == cls_pred)

    correct_sum = correct.sum()

    acc = float(correct_sum) / num_test

    msg = "Validation Accuracy: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))

    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)

def print_test_accuracy(show_example_errors=False, show_confusion_matrix=False):

    num_test2 = len(data3.test.images)

    cls_pred2 = np.zeros(shape=num_test2, dtype=np.int)

    i = 0

    while i < num_test2:
 
        j = min(i + batch_size, num_test2)

        images = data3.test.images[i:j, :].reshape(-1, img_size_flat)
        
        labels = data3.test.labels[i:j, :]

        feed_dict = {x: images, y_true: labels}

        cls_pred2[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        i = j


    cls_true2 = np.array(data3.test.cls)
    cls_pred2 = np.array([classes[x] for x in cls_pred2])

 

    correct = (cls_true2 == cls_pred2)

    correct_sum = correct.sum()

    acc = float(correct_sum) / num_test2

    msg = "Test Accuracy: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test2))

    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred2=cls_pred2, correct=correct)

    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix2(cls_pred2=cls_pred2)


def plot_confusion_matrix(cls_pred):
    
    cls_true = data2.valid.cls

    cm = confusion_matrix(y_true=cls_true, y_pred=cls_pred)
    
    print(cm)
    
    plt.matshow(cm, cmap=plt.cm.Wistia_r)

    plt.colorbar()
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, range(num_classes))
    plt.yticks(tick_marks, range(num_classes))
    plt.grid(False)

    plt.xlabel('Predicted')
    plt.ylabel('True')
    for i in range(0,4):
      for j in range(0,4):
        plt.text(j,i,str(cm[i][j]))
    plt.show()

    print("1.Cancer의 정밀도 : ", (cm[0,0]/(cm[0,0]+cm[1,0]+cm[2,0]+cm[3,0]))*100, "%")
    print("1.Cancer의 재현율 : ", (cm[0,0]/(cm[0,0]+cm[0,1]+cm[0,2]+cm[0,3]))*100, "%")
    print("2.Precancer의 정밀도 : ", (cm[1,1]/(cm[0,1]+cm[1,1]+cm[2,1]+cm[3,1]))*100, "%")
    print("2.Precancer의 재현율 : ", (cm[1,1]/(cm[1,0]+cm[1,1]+cm[1,2]+cm[1,3]))*100, "%")
    print("3.Inflammatory의 정밀도 : ", (cm[2,2]/(cm[0,2]+cm[1,2]+cm[2,2]+cm[3,2]))*100, "%")
    print("3.Inflammatory의 재현율 : ", (cm[2,2]/(cm[2,0]+cm[2,1]+cm[2,2]+cm[2,3]))*100, "%")
    print("4.Normal의 정밀도 : ", (cm[3,3]/(cm[0,3]+cm[1,3]+cm[2,3]+cm[3,3]))*100, "%")
    print("4.Normal의 재현율 : ", (cm[3,3]/(cm[3,0]+cm[3,1]+cm[3,2]+cm[3,3]))*100, "%")


def plot_confusion_matrix2(cls_pred2):
    
    cls_true2 = data3.test.cls

    cm = confusion_matrix(y_true=cls_true2, y_pred=cls_pred2)
    
    print(cm)
    
    plt.matshow(cm, cmap=plt.cm.Wistia_r)

    plt.colorbar()
    
    plt.title('Test Confusion Matrix')
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, range(num_classes))
    plt.yticks(tick_marks, range(num_classes))
    plt.grid(False)

    plt.xlabel('Predicted')
    plt.ylabel('True')
    for i in range(0,4):
      for j in range(0,4):
        plt.text(j,i,str(cm[i][j]))
    plt.show()

    print("1.Cancer의 정밀도 : ", (cm[0,0]/(cm[0,0]+cm[1,0]+cm[2,0]+cm[3,0]))*100, "%")
    print("1.Cancer의 재현율 : ", (cm[0,0]/(cm[0,0]+cm[0,1]+cm[0,2]+cm[0,3]))*100, "%")
    print("2.Precancer의 정밀도 : ", (cm[1,1]/(cm[0,1]+cm[1,1]+cm[2,1]+cm[3,1]))*100, "%")
    print("2.Precancer의 재현율 : ", (cm[1,1]/(cm[1,0]+cm[1,1]+cm[1,2]+cm[1,3]))*100, "%")
    print("3.Inflammatory의 정밀도 : ", (cm[2,2]/(cm[0,2]+cm[1,2]+cm[2,2]+cm[3,2]))*100, "%")
    print("3.Inflammatory의 재현율 : ", (cm[2,2]/(cm[2,0]+cm[2,1]+cm[2,2]+cm[2,3]))*100, "%")
    print("4.Normal의 정밀도 : ", (cm[3,3]/(cm[0,3]+cm[1,3]+cm[2,3]+cm[3,3]))*100, "%")
    print("4.Normal의 재현율 : ", (cm[3,3]/(cm[3,0]+cm[3,1]+cm[3,2]+cm[3,3]))*100, "%")

In [ ]:
optimize(num_iterations=1000000)